# Merge & Analysis

In [ ]:
library(data.table)
library(stringr)
library(ggplot2)
library(psych)

In [ ]:
oneOBigger <- function(nn){
    if (nn > 0) {
        return(nn)
    } else {
        return(1)
    }
}
duration <- function(myvar, window = 5) {
    
    durc <- integer(nrow(myvar))
    for (current_line in 1:nrow(myvar)){
            current_auth_both <- union(myvar$s_author[current_line], myvar$t_author[current_line])
            past_auth_in_window <- myvar[c((oneOBigger(current_line-window)):current_line),c("s_author","t_author")]
            durc[current_line] <- sum(is.element(past_auth_in_window$s_author, current_auth_both) & is.element(past_auth_in_window$t_author, current_auth_both))
        }
     return(durc)
}

In [ ]:
received_negativity <- function(df_test) {
    myvar <- copy(df_test)
    durc <- integer(nrow(myvar))
    for (current_line in 1:nrow(myvar)){
         
        current_target = myvar$t_revision[current_line]
        current_s_author = myvar$s_author[current_line]
        
        set1 <- myvar[myvar$s_revision == current_target & myvar$t_author == current_s_author,]
        
        durc[current_line] <- (sum(str_count(set1$deleted...out, "'")) / 2) + ( sum(str_count(set1$undo_delete...in, "'")) / 2) +  ( sum(str_count(set1$undo_reintro...out, "'")) / 2)
        }
     return(durc)
}

In [ ]:
reciprocity <- function(df_test) {
    myvar <- copy(df_test)
    durc <- integer(nrow(myvar))
    for (current_line in 1:nrow(myvar)){
         
        current_target = myvar$t_revision[current_line]
        current_s_author = myvar$s_author[current_line]
        
        set1 <- myvar[myvar$s_revision == current_target & myvar$t_author == current_s_author,]
        
        zzz <- (sum(str_count(set1$deleted...out, "'")) / 2) + ( sum(str_count(set1$undo_delete...in, "'")) / 2) +  ( sum(str_count(set1$undo_reintro...out, "'")) / 2)
        iii <- (sum(str_count(myvar[current_line,]$deleted...out, "'")) / 2) + ( sum(str_count(myvar[current_line,]$undo_delete...in, "'")) / 2) +  ( sum(str_count(myvar[current_line,]$undo_reintro...out, "'")) / 2)
             
        if (zzz != 0) {
            durc[current_line] = (iii) / (zzz)
        }
        if (durc[current_line] > 1){
            durc[current_line] <- 1
        }
    }
     return(durc)
}

In [ ]:
data <- read.csv("data/Eire_fabout.csv", sep="\t", stringsAsFactors = F)
head(data)

In [ ]:
data['duration'] <- duration(data, window = 30)

In [ ]:
data['received_negativity'] <- received_negativity(data)

In [ ]:
data['reciprocity'] <- reciprocity(data)

In [ ]:
analysis <- data[c('duration', 'received_negativity', 'reciprocity', 'negativity', 'negativity_noadds','ratio_swords')]

In [ ]:
head(analysis)

In [ ]:
plot(analysis)

In [ ]:
cor(analysis)

In [ ]:
summary(analysis)

In [ ]:
hist(analysis$duration)

In [ ]:
hist(analysis$received_negativity)

In [ ]:
hist(analysis$reciprocity)

In [ ]:
hist(analysis$negativity)

In [ ]:
hist(analysis$negativity_noadds)

In [ ]:
hist(analysis$ratio_swords)

In [ ]:
# write.csv(data, "data_all_features.1.csv")

> ggplot(analysis, aes(x = received_negativity, y = reciprocity, )) + 
  geom_point() +
scale_x_continuous(trans='log2') +
  scale_y_continuous(trans='log2')